In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Taipei Housing Price Prediction") \
    .getOrCreate()


### 타이베이 주택 가격 예측 모델 만들기

데이터셋 설명

이번 문제는 대만 타이베이 시의 신단 지역에서 수집된 주택 거래 관련 정보를 바탕으로 주택 가격(정확히는 주택의 평당 가격)을 예측하는 Regression 모델을 만들어보는 것이다. 총 6개의 피쳐와 주택의 평당 가격에 해당하는 레이블 정보가 훈련 데이터로 제공된다. 레이블의 경우에는 주택의 최종 가격이 아니라 평당 가격이란 점을 다시 한번 강조한다.

각 컬럼에 대한 설명은 아래와 같으며 모든 필드는 X4를 제외하고는 실수 타입이다.

X1: 주택 거래 날짜를 실수로 제공한다. 소수점 부분은 달을 나타낸다. 예를 들어 2013.250이라면 2013년 3월임을 나타낸다 (0.250 = 3/12)
X2: 주택 나이 (년수)
X3: 가장 가까운 지하철역까지의 거리 (미터)
X4: 주택 근방 걸어갈 수 있는 거리내 편의점 수
X5: 주택 위치의 위도 (latitude)
X6: 주택 위치의 경도 (longitude)
Y: 주택 평당 가격

In [116]:
data = spark.read.csv('data/Taipei_sindan_housing.csv', header=True, inferSchema=True)
data.printSchema()
data.show()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: integer (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: double (nullable = true)
 |-- Y: double (nullable = true)

+--------+----+--------+---+--------+---------+----+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|
+--------+----+--------+---+--------+---------+----+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|
|2013.083|34.8|405.2134|  1|24.97349|121.53372|41.4|
|2013.333| 6.3

### 데이터 전처리

In [117]:
from pyspark.sql.functions import floor, round, col

data = data.withColumn("Year", floor(data["X1"]))
data = data.withColumn("Month", round((data["X1"] - floor(data["X1"])) * 12))
data = data.drop("X1")  # 원래 X1은 제거

# Y 컬럼을 뒤로 빼기 위해 나머지 컬럼 선택 후 Y 추가
data = data.select([col(c) for c in data.columns if c != "Y"] + [col("Y")])


### 피쳐 벡터를 만들기

In [119]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="feature")
data_2 = assembler.transform(data)
data_2.show()

+----+--------+---+--------+---------+----+-----+----+--------------------+
|  X2|      X3| X4|      X5|       X6|Year|Month|   Y|             feature|
+----+--------+---+--------+---------+----+-----+----+--------------------+
|32.0|84.87882| 10|24.98298|121.54024|2012| 11.0|37.9|[32.0,84.87882,10...|
|19.5|306.5947|  9|24.98034|121.53951|2012| 11.0|42.2|[19.5,306.5947,9....|
|13.3|561.9845|  5|24.98746|121.54391|2013|  7.0|47.3|[13.3,561.9845,5....|
|13.3|561.9845|  5|24.98746|121.54391|2013|  6.0|54.8|[13.3,561.9845,5....|
| 5.0|390.5684|  5|24.97937|121.54245|2012| 10.0|43.1|[5.0,390.5684,5.0...|
| 7.1| 2175.03|  3|24.96305|121.51254|2012|  8.0|32.1|[7.1,2175.03,3.0,...|
|34.5|623.4731|  7|24.97933|121.53642|2012|  8.0|40.3|[34.5,623.4731,7....|
|20.3|287.6025|  6|24.98042|121.54228|2013|  5.0|46.7|[20.3,287.6025,6....|
|31.7|5512.038|  1|24.95095|121.48458|2013|  6.0|18.8|[31.7,5512.038,1....|
|17.9| 1783.18|  3|24.96731|121.51486|2013|  5.0|22.1|[17.9,1783.18,3.0...|
|34.8|405.21

### 훈령용과 테스트용 데이터를 나누기

In [120]:
train, test = data_2.randomSplit([0.7, 0.3])

### 피쳐 스케일링

In [121]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="feature", outputCol="scaled_features", withMean=True, withStd=True)
scaler_model = scaler.fit(train)
train = scaler_model.transform(train)
test = scaler_model.transform(test)

### 모델 학습

In [122]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="scaled_features", labelCol="Y")
model = algo.fit(train)

### 모델 성능 측정

In [123]:
evaluation_summary = model.evaluate(test)

print(evaluation_summary.meanAbsoluteError)
print(evaluation_summary.rootMeanSquaredError)
print(evaluation_summary.r2)

4.9175855214366395
6.040877085623429
0.7458366458090393


### 모델 예측값 살펴보기

In [124]:
predictions = model.transform(test)

In [125]:
predictions.select(predictions.columns[7:]).show()

+----+--------------------+--------------------+------------------+
|   Y|             feature|     scaled_features|        prediction|
+----+--------------------+--------------------+------------------+
|52.2|[0.0,274.0144,1.0...|[-1.5256889692580...|45.428423877096485|
|69.7|[0.0,292.9978,6.0...|[-1.5256889692580...| 51.50949264509884|
|45.1|[1.1,193.5845,6.0...|[-1.4310940818528...| 48.85777330535033|
|48.6|[1.1,193.5845,6.0...|[-1.4310940818528...| 48.85777330535033|
|49.7|[1.5,23.38284,7.0...|[-1.3966959409782...| 47.92812992444633|
|50.4|[1.7,329.9747,5.0...|[-1.3794968705409...|50.932073445626436|
|27.0|[1.8,1455.798,1.0...|[-1.3708973353222...|30.961068595258965|
|33.4|[2.0,2077.39,3.0,...|[-1.3536982648849...|33.699976733754504|
|45.5|[2.1,451.2438,5.0...|[-1.3450987296662...| 45.70580439077242|
|45.4|[2.3,184.3302,6.0...|[-1.3278996592289...| 48.84550210501892|
|36.9|[2.5,156.2442,4.0...|[-1.3107005887916...| 46.61280942085778|
|31.1|[2.6,1554.25,3.0,...|[-1.3021010535730...|

### 랜덤포레스트

In [126]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# 랜덤 포레스트 모델 학습
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="Y", numTrees=100)
rf_model = rf.fit(train)

# 예측 수행
rf_predictions = rf_model.transform(test)

# 평가 지표 계산
evaluator_mae = RegressionEvaluator(labelCol="Y", predictionCol="prediction", metricName="mae")
evaluator_rmse = RegressionEvaluator(labelCol="Y", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="Y", predictionCol="prediction", metricName="r2")

# 결과 출력
print("RF MAE:", evaluator_mae.evaluate(rf_predictions))
print("RF RMSE:", evaluator_rmse.evaluate(rf_predictions))
print("RF R²:", evaluator_r2.evaluate(rf_predictions))

RF MAE: 4.072797193777875
RF RMSE: 5.187665871661255
RF R²: 0.8125623083211783


In [ ]:
model.save("model/Taipei_LR_model")

In [129]:
rf_model.save("model/Taipei_RF_model")

### 모델 불러오기

In [131]:
from pyspark.ml.regression import RandomForestRegressionModel
loaded_model = RandomForestRegressionModel.load("model/Taipei_RF_model")

In [133]:
prediction2 = loaded_model.transform(test)

In [136]:
prediction2.select(prediction2.columns[7:]).show()

+----+--------------------+--------------------+------------------+
|   Y|             feature|     scaled_features|        prediction|
+----+--------------------+--------------------+------------------+
|52.2|[0.0,274.0144,1.0...|[-1.5256889692580...|51.333516452022366|
|69.7|[0.0,292.9978,6.0...|[-1.5256889692580...| 62.21257512026732|
|45.1|[1.1,193.5845,6.0...|[-1.4310940818528...| 49.98573788671144|
|48.6|[1.1,193.5845,6.0...|[-1.4310940818528...| 49.98573788671144|
|49.7|[1.5,23.38284,7.0...|[-1.3966959409782...| 50.40535872333774|
|50.4|[1.7,329.9747,5.0...|[-1.3794968705409...|56.963567984845895|
|27.0|[1.8,1455.798,1.0...|[-1.3708973353222...|26.405026893043836|
|33.4|[2.0,2077.39,3.0,...|[-1.3536982648849...|27.406967211664732|
|45.5|[2.1,451.2438,5.0...|[-1.3450987296662...| 49.23772623147784|
|45.4|[2.3,184.3302,6.0...|[-1.3278996592289...| 50.28719585405772|
|36.9|[2.5,156.2442,4.0...|[-1.3107005887916...| 48.92075508891202|
|31.1|[2.6,1554.25,3.0,...|[-1.3021010535730...|